# 🏠 مشروع: التنبؤ بدرجة الصحة (Health Score Prediction) — بصيغة المحاضرة
**وصف:** مشروع منظّم خطوة بخطوة لتنبؤ “Health Score” باستخدام نماذج انحدار: Decision Tree, Random Forest, XGBoost, LightGBM.


**ملاحظات قبل التشغيل:**
- كل خلية مستقلة ويمكن تشغيلها على حدة.
- الشرح بالعربية مع كود قابل للتنفيذ في Python (Jupyter / Colab).
- ستحصل على ملف `best_model.pkl` و`scaler.pkl` و`encoders.pkl` و`regression_models_results.csv` بعد تشغيل الخلايا النهائية.

---


## 🛠️ إعداد البيئة (Environment Setup)
شرح: استيراد المكتبات الأساسية وضبط البيئة. شغّل هذه الخلية أولًا.

In [ ]:
# -*- coding: utf-8 -*-
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from pathlib import Path

# مكتبات scikit-learn الأساسية
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error

# نماذج
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import randint, uniform

np.random.seed(42)

# مكتبات اختيارية (XGBoost, LightGBM)
try:
    from xgboost import XGBRegressor
except Exception:
    XGBRegressor = None
    print('⚠️ xgboost غير مثبت — لتثبيت: pip install xgboost')

try:
    from lightgbm import LGBMRegressor
except Exception:
    LGBMRegressor = None
    print('⚠️ lightgbm غير مثبت — لتثبيت: pip install lightgbm')

## 📥 تحميل البيانات (Load Data)
شرح: ضَع ملف البيانات `synthetic_health_data.csv` في نفس مجلد الـ Notebook، ثم شغّل الخلية لتحميل المعاينة.

In [ ]:
DATA_PATH = Path("synthetic_health_data.csv")

if not DATA_PATH.exists():
    raise FileNotFoundError(f"❌ لم أجد ملف البيانات: {DATA_PATH.resolve()}\nأرفِع الملف أو حدّث مسار DATA_PATH.")

df = pd.read_csv(DATA_PATH)
print("✅ تم تحميل البيانات — الشكل:", df.shape)
display(df.head())

## 🧠 تجهيز الميزات (Feature Engineering)
شرح: إذا لم يكن عمود `Health_Score` موجودًا، سننشئه من مجموعة من الأعمدة الصحية باستخدام أوزان مبدئية قابلة للتعديل. بعدها نعرض وصفًا إحصائيًا للبيانات.

In [ ]:
# إنشاء Health_Score إذا لم يكن موجودًا
if "Health_Score" not in df.columns:
    print("⚙️ إنشاء عمود Health_Score تلقائيًا باستخدام أوزان مبدئية...")
    # يمكنك تعديل هذه الأوزان بحسب خبرتك أو دراسة المجال
    weights = {
        "Sleep_Hours": 0.20,
        "Diet_Quality": 0.25,
        "Activity_Level": 0.25,
        "Stress_Level": -0.15,
        "Blood_Pressure": -0.10,
        "Age": -0.05
    }
    # التحقق من وجود الأعمدة المطلوبة
    missing = [c for c in weights.keys() if c not in df.columns]
    if missing:
        raise KeyError(f"الأعمدة التالية مفقودة لإنشاء Health_Score: {missing}")
    df["Health_Score"] = sum(df[col] * w for col, w in weights.items())
    # إعادة تحجيم إلى نطاق 0-100
    df["Health_Score"] = 100 * (df["Health_Score"] - df["Health_Score"].min()) / (df["Health_Score"].max() - df["Health_Score"].min())
else:
    print("ℹ️ عمود Health_Score موجود بالفعل — سنستخدمه كما هو.")

display(df.describe().T)

## 📊 التحليل الاستكشافي (EDA)
شرح: سنستعمل رسومات توضيحية لفهم العلاقات والتوزيع. (ملاحظة: نستخدم matplotlib لعرض الرسوم لضمان التوافق مع بيئات مختلفة).

In [ ]:
# مصفوفة الارتباط (correlation matrix) ورسم خريطة حرارة باستخدام matplotlib
corr = df.corr()

fig, ax = plt.subplots(figsize=(10, 8))
cax = ax.matshow(corr, cmap='coolwarm')
fig.colorbar(cax)
ax.set_xticks(range(len(corr.columns)))
ax.set_yticks(range(len(corr.columns)))
ax.set_xticklabels(corr.columns, rotation=90)
ax.set_yticklabels(corr.columns)
plt.title("خريطة الارتباط بين المتغيرات", pad=20)
plt.show()

# توزيع الهدف
fig, ax = plt.subplots(figsize=(8,4))
ax.hist(df['Health_Score'].dropna(), bins=30)
ax.set_title('توزيع Health_Score')
ax.set_xlabel('Health_Score')
ax.set_ylabel('العدد')
plt.show()

# رسم علاقة بين بعض المتغيرات المهمة و Health_Score
pairs = [('Sleep_Hours', 'Health_Score'), ('Activity_Level', 'Health_Score'), ('Stress_Level', 'Health_Score')]
for xcol, ycol in pairs:
    if xcol in df.columns and ycol in df.columns:
        fig, ax = plt.subplots(figsize=(6,4))
        ax.scatter(df[xcol], df[ycol], alpha=0.6)
        ax.set_xlabel(xcol)
        ax.set_ylabel(ycol)
        ax.set_title(f'{ycol} مقابل {xcol}')
        plt.show()

## ✂️ الترميز (Encoding) والقياس (Scaling)
شرح: ترميز الأعمدة النصية ثم قياس الميزات باستخدام StandardScaler. سنحفظ الـ encoders و scaler لاستخدامها لاحقًا عند النشر.

In [ ]:
TARGET = 'Health_Score'
X = df.drop(columns=[TARGET])
y = df[TARGET].astype(float)

encoders = {}
for col in X.columns:
    if X[col].dtype == 'object':
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
        encoders[col] = le

# ملء القيم الناقصة البسيط (إن وجدت) — هنا نعوض بالمتوسط للميزات العددية
X = X.fillna(X.mean())

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# حفظ مؤقت للـ encoders (لا تقم بالحفظ النهائي حتى نعرف أفضل نموذج)
# joblib.dump(encoders, 'encoders_partial.pkl')
print('✅ الترميز والقياس جاهزان. شكل X:', X.shape)

## 🔀 تقسيم البيانات (Train/Test Split)
شرح: نقسم البيانات إلى مجموعة تدريب ومجموعة اختبار (80/20).

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)
print('✅ Training shape:', X_train.shape, 'Testing shape:', X_test.shape)

## ⚙️ تدريب النماذج وضبط المعاملات (Hyperparameter Tuning)
شرح: سنستخدم `RandomizedSearchCV` لكل نموذج — الفراغات هنا أوسع قليلًا لتعكس منهج المحاضرة.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score, mean_absolute_error

def run_random_search(name, estimator, param_dist, n_iter=30, cv=3):
    print(f"\n🔧 بدء الضبط: {name}")
    rs = RandomizedSearchCV(estimator, param_distributions=param_dist, n_iter=n_iter,
                            scoring='r2', cv=cv, random_state=42, n_jobs=-1, verbose=0)
    rs.fit(X_train, y_train)
    best = rs.best_estimator_
    y_pred = best.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"✅ {name} — R2_test: {r2:.4f} | MAE_test: {mae:.4f}")
    print('🔧 أفضل معاملات (best_params):', rs.best_params_)
    return {'name': name, 'model': best, 'r2': r2, 'mae': mae, 'best_params': rs.best_params_}

In [ ]:
results = []

# Decision Tree
dt_params = {
    'max_depth': [3, 5, 8, 12, None],
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 6),
    'criterion': ['squared_error', 'friedman_mse']
}
results.append(run_random_search('DecisionTree', DecisionTreeRegressor(random_state=42), dt_params, n_iter=30))

# Random Forest
rf_params = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [5, 8, 12, None],
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 6),
    'max_features': ['auto', 'sqrt', 'log2']
}
results.append(run_random_search('RandomForest', RandomForestRegressor(random_state=42), rf_params, n_iter=30))

# XGBoost (if available)
if XGBRegressor is not None:
    xgb_params = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 8],
        'learning_rate': [0.01, 0.05, 0.1],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0]
    }
    results.append(run_random_search('XGBoost', XGBRegressor(eval_metric='rmse', random_state=42), xgb_params, n_iter=30))
else:
    print('⚠️ تخطي XGBoost — لم يتم العثور على المكتبة.')

# LightGBM (if available)
if LGBMRegressor is not None:
    lgb_params = {
        'n_estimators': [100, 200, 300],
        'num_leaves': [15, 31, 63],
        'learning_rate': [0.01, 0.05, 0.1],
        'min_data_in_leaf': [10, 20, 40]
    }
    results.append(run_random_search('LightGBM', LGBMRegressor(random_state=42), lgb_params, n_iter=30))
else:
    print('⚠️ تخطي LightGBM — لم يتم العثور على المكتبة.')

## 📈 تقييم النماذج ومقارنة الأداء
شرح: نلخّص النتائج في جدول ونرسم مقارنات R² و MAE.

In [ ]:
res_df = pd.DataFrame([{
    'Model': r['name'],
    'R2_test': r['r2'],
    'MAE_test': r['mae'],
    'best_params': r['best_params']
} for r in results])

res_df = res_df.sort_values('R2_test', ascending=False).reset_index(drop=True)
display(res_df[['Model','R2_test','MAE_test']])

# رسم مقارنة R2
fig, ax = plt.subplots(figsize=(8,5))
ax.bar(res_df['Model'], res_df['R2_test'])
ax.set_ylabel('R2 (Test)')
ax.set_title('مقارنة النماذج حسب R2')
plt.show()

# رسم مقارنة MAE
fig, ax = plt.subplots(figsize=(8,5))
ax.bar(res_df['Model'], res_df['MAE_test'])
ax.set_ylabel('MAE (Test)')
ax.set_title('مقارنة النماذج حسب MAE')
plt.show()

## 🏆 اختيار الأفضل وحفظه
شرح: اختيار النموذج الأعلى R2 على الاختبار ثم حفظ النموذج والمحوّل والترميزات ونتائج الضبط.

In [ ]:
best_row = res_df.loc[res_df['R2_test'].idxmax()]
best_name = best_row['Model']
best_model_obj = next(r['model'] for r in results if r['name'] == best_name)

print(f"🏆 أفضل نموذج: {best_name} — R2_test = {best_row['R2_test']:.4f}")

joblib.dump(best_model_obj, 'best_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(encoders, 'encoders.pkl')
res_df.to_csv('regression_models_results.csv', index=False)
print('✅ حفظت: best_model.pkl, scaler.pkl, encoders.pkl, regression_models_results.csv')

## 📝 الخلاصة (Summary & Insights)
- لخص هنا النتائج الرئيسية: أي نموذج تفوق، قيم R² وMAE، وما المعاملات المهمة التي اكتشفناها.
- اذكر توصيات عملية: مثل جمع بيانات إضافية، ميزات جديدة متوقعة التأثير، أو خطوات لتحسين الأداء (feature engineering, ensemble, stacking).

## 🚀 تحضير للنشر (Deployment Preparation)
شرح مختصر: يمكنك نشر `best_model.pkl` مع `scaler.pkl` و`encoders.pkl` عبر واجهة بسيطة باستخدام Streamlit أو Flask. فيما يلي مثال مبسط لكود Streamlit للنشر:

```python
# streamlit_app.py
import streamlit as st
import joblib
import pandas as pd

model = joblib.load('best_model.pkl')
scaler = joblib.load('scaler.pkl')
encoders = joblib.load('encoders.pkl')

st.title('Health Score Predictor')
# ... إضافة واجهة مدخلات المستخدم ثم التنبؤ
```

— انتهى المشروع.
